# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [17]:
# Imports here
import sys
import os
sys.path.append("/Users/silvas/training/udacity/ml_intro_pytorch/github/DSND_Term1/projects/p2_image_classifier")
#print (sys.path)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
from numpy import ndarray
import torch
import math
from math import floor

import helper

import matplotlib.pyplot as plt

###

from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms, models

import helper

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [18]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [19]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#When you're testing however, you'll want to use images that aren't altered other than normalizing. So, for validation/test images, you'll typically just resize and crop.
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_datasets = datasets.ImageFolder(train_dir, transform=train_transforms)
test_datasets = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_datasets = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_datasets, batch_size=32, shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [5]:
# LOAD PRETRAINED MODEL
model = models.vgg16(pretrained=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:50<00:00, 10943679.95it/s]


In [6]:
## BUILD CLASSIFIER

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict(
            [
                ('fc1', nn.Linear(25088, 10000)),
                ('relu', nn.ReLU()),
                ('drop', nn.Dropout(0.2)),
                ('fc2', nn.Linear(10000, 2000)),
                ('relu', nn.ReLU()),
                ('drop', nn.Dropout(0.2)),
                ('fc3', nn.Linear(2000, 500)),
                ('relu', nn.ReLU()),
                ('drop', nn.Dropout(0.2)),
                ('fc4', nn.Linear(500, 102)),
                ('output', nn.LogSoftmax(dim=1))
            ]))
         
model.classifier = classifier
#print(model.classifier.in_features)
#print(model.fc.in_features)
print(model.classifier)

features = list(model.classifier.children())[:-1]
#num_filters = model.classifier[len(features)].in_features


criterion = nn.NLLLoss()
# Optimize only classifier parameters, not model features parameters
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001) # 0.002 (yes, 20x bigger) was causing accuracy fluctuation on validation set
'''
# Only train the classifier parameters, feature parameters are frozen
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
optimizer = optim.Adam(params_to_update, lr = 0.001)
'''

model.to(device)
         
for images, labels in trainloader:
    print(images.shape) # torch.Size([32, 3, 224, 224]) So, 32 images per batch, 3 color channel, and 224x224 images.
    print(labels.shape) # torch.Size([32])
    break
    
'''    
image, label = next(iter(trainloader))
helper.imshow(image[0,:]);
helper.view_classify(img.resize_(1, 28, 28), ps, version='Fashion')
'''

Sequential(
  (fc1): Linear(in_features=25088, out_features=10000, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.2)
  (fc2): Linear(in_features=10000, out_features=2000, bias=True)
  (fc3): Linear(in_features=2000, out_features=500, bias=True)
  (fc4): Linear(in_features=500, out_features=102, bias=True)
  (output): LogSoftmax()
)
torch.Size([32, 3, 224, 224])
torch.Size([32])


"    \nimage, label = next(iter(trainloader))\nhelper.imshow(image[0,:]);\nhelper.view_classify(img.resize_(1, 28, 28), ps, version='Fashion')\n"

In [7]:
# TRAIN

epochs = 5
steps = 0
running_loss = 0
print_every = 10

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # images = images.view(images.shape[0], -1) # no need of reshaping...why?
        steps += 1
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # outputs = model.forward(inputs)
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Validation loop -but then why use testloader and not validloader??
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            with torch.no_grad():
                model.eval()
                for inputs, labels in validloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {e+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(validloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(validloader):.3f}")
            running_loss = 0
            model.train()
    '''    
    else:
        print(f"Training loss: {running_loss} ")
    ''' 
        

Epoch 1/5.. Train loss: 4.322.. Test loss: 4.053.. Test accuracy: 0.089
Epoch 1/5.. Train loss: 3.731.. Test loss: 3.307.. Test accuracy: 0.316
Epoch 1/5.. Train loss: 3.122.. Test loss: 2.569.. Test accuracy: 0.451
Epoch 1/5.. Train loss: 2.371.. Test loss: 2.050.. Test accuracy: 0.523
Epoch 1/5.. Train loss: 2.007.. Test loss: 1.774.. Test accuracy: 0.561
Epoch 1/5.. Train loss: 1.704.. Test loss: 1.432.. Test accuracy: 0.651
Epoch 1/5.. Train loss: 1.481.. Test loss: 1.191.. Test accuracy: 0.695
Epoch 1/5.. Train loss: 1.309.. Test loss: 1.175.. Test accuracy: 0.680
Epoch 1/5.. Train loss: 1.167.. Test loss: 1.079.. Test accuracy: 0.714
Epoch 1/5.. Train loss: 1.121.. Test loss: 0.921.. Test accuracy: 0.748
Epoch 1/5.. Train loss: 0.981.. Test loss: 0.892.. Test accuracy: 0.762
Epoch 1/5.. Train loss: 0.865.. Test loss: 0.788.. Test accuracy: 0.800
Epoch 1/5.. Train loss: 0.770.. Test loss: 0.675.. Test accuracy: 0.819
Epoch 1/5.. Train loss: 0.814.. Test loss: 0.758.. Test accuracy

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [28]:
# Test out your network!
'''
dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[1]

# TODO: Calculate the class probabilities (softmax) for img
ps = torch.exp(model(img))

# Plot the image and probabilities
helper.view_classify(img, ps, version='Fashion')
# Make sure the shape is appropriate, we should get 10 class probabilities for 64 examples
print(ps.shape)
'''

test_loss = 0
accuracy = 0
with torch.no_grad():
    model.eval()
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)

        test_loss += batch_loss.item()

        # Calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test loss: {test_loss/len(testloader):.3f}.. "
      f"Test accuracy: {accuracy/len(testloader):.3f}")

model.train()

Test loss: 0.432.. Test accuracy: 0.893


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [10]:
# TODO: Do validation on the test set
# Done in the training

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [29]:
# TODO: Save the checkpoint

def save_checkpoint(filepath):
    # https://pytorch.org/tutorials/beginner/saving_loading_models.html
    '''
    model.class_to_idx = image_datasets['train'].class_to_idx
    model.epochs = epochs
    model.optimizer_state_dict = optimizer.state_dict
    
    checkpoint = {'input_size': 25088,
          'output_size': 102,
          'hidden_layers': [each.out_features for each in model.hidden_layers],
          'state_dict': model.state_dict() # includes parameters set before 'checkpoint' ?
            }
    '''
    torch.save(model, filepath)

save_checkpoint('checkpoint.pth')


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [3]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    '''
    checkpoint = torch.load(filepath)
    print(checkpoint.keys())
    ### model = models.vgg16(pretrained=True)
    ## How do I instantiate 'model'?
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    '''
    model = torch.load(filepath)
    return model

model = load_checkpoint('checkpoint.pth')
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [72]:
# TODO: Process a PIL image for use in a PyTorch model
def process_image(image):
    ''' 
    Scales, crops, and normalizes a PIL image for a PyTorch model,
    returns an Numpy array
    '''
    import PIL
    from PIL import Image
    
    '''
    im = Image.open(image)
    #im.rotate(45).show()
    #im = im.resize((256, 256)) # i'm not keeping the aspect ratio...
    im = im.thumbnail([256, 256],PIL.Image.ANTIALIAS) 
    print(f" IM SIZE: {im.size}")
    im = im.crop((224, 224,224,224))
    '''
    
    image = Image.open(image)
    width, height = image.size
    size = 256, 256
    ratio = float(width) / float(height)
    
    if width > height:
        newheight = ratio * size[0]
        image = image.resize((size[0], int(floor(newheight))), Image.ANTIALIAS)
    else:
        newwidth = size[1] / ratio
        image = image.resize((int(floor(newwidth)), size[1]), Image.ANTIALIAS)
    size = image.size
    print(f"SIZE: {size}")
    image.show()
    # See this for better understanding of crop() -> https://stackoverflow.com/questions/20361444/cropping-an-image-with-python-pillow
    # It starts counting on top left corner... not in the center
    image = image.crop((
        size[0] / 2 - (224/2),
        size[1] / 2 - (224/2),
        size[0] / 2 + (224/2),
        size[1] / 2 + (224/2),
    ))
    
    print(size[0] / 2 - (224/2))
    print(f"SIZE AFTER CROP: {image.size}")

    # convert colors
    np_image = np.array(image)
    im = np_image.astype('float64') / 255 # convert to [0,1]
    
    # normalize
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    im = (im - mean) / std
    
    im = im.transpose((2,0,1)) # The color channel needs to be first and retain the order of the other two dimensions.
    
    return im

    '''
    probs, classes = predict(image_path, model)
    print(probs)
    print(classes)
    > [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
    > ['70', '3', '45', '62', '55']
    '''

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [73]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [78]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    image = process_image(image_path)
    #image = imshow(image)
    
    ps = torch.exp(model(torch.from_numpy(image))) # need to convert to pytorch tensor...
    print(ps.shape)
    top_p, top_class = ps.topk(topk, dim=1)
    # Look at the most likely classes for the first 10 examples
    print(top_class)
    
    top_flowers = []
    for i in top_class:
        top_flowers.append(i.class_to_idx)
        
    return top_p, top_flowers
    
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [79]:
# TODO: Display an image along with the top 5 classes
probs, classes = predict('flowers/test/1/image_06743.jpg', model)
print(probs)
print(classes)

imshow(image_path)


plt.xlabel('Prob', weight='semibold')
plt.ylabel('Class', weight='semibold')

for i in range(len(classes)):
    bars = plt.bar(i)

plt.yticks(classes)

plt.show()

fig, ax = plt.subplots()

# Example data
y_pos = np.arange(len(classes))
performance = 3 + 10 * np.random.rand(len(classes))
probs = np.random.rand(len(people))

ax.barh(y_pos, probs, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(classes)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Probs')
ax.set_title('Top-K')

SIZE: (307, 256)
41.5
SIZE AFTER CROP: (224, 224)


RuntimeError: expected stride to be a single integer value or a list of 1 values to match the convolution dimensions, but got stride=[1, 1]